In [1]:
import os
import sys

In [2]:
sys.path.append('../lib')
sys.path.append('../lib/sigkernel')

In [3]:
import cov_independence_test
import hsic_independence_test
import gen_processes

In [4]:
from importlib import reload
import pickle

In [5]:
reload(cov_independence_test);
reload(hsic_independence_test);
reload(gen_processes);

In [6]:
import datetime
import logging
import logging.handlers
LOGGER = logging.getLogger('logger')

def configure_logger():
    warning_handler = logging.StreamHandler()
    warning_handler.setLevel('WARNING')

    info_handler = logging.StreamHandler(sys.stdout)
    info_handler.setLevel('DEBUG')
    info_handler.addFilter(lambda record: record.levelno <= logging.INFO)

    LOGGER.addHandler(warning_handler)
    LOGGER.addHandler(info_handler)

def add_file_handlers(warning_log, info_log, debug_log):
    warning_log_handler = logging.FileHandler(warning_log)
    warning_log_handler.setLevel('WARNING')

    info_log_handler = logging.FileHandler(info_log)
    info_log_handler.addFilter(lambda record: record.levelno <= logging.INFO)
    info_log_handler.setLevel('INFO')

    debug_log_handler = logging.FileHandler(debug_log)
    debug_log_handler.addFilter(lambda record: record.levelno <= logging.DEBUG)
    debug_log_handler.setLevel('DEBUG')

    LOGGER.addHandler(warning_log_handler)
    LOGGER.addHandler(info_log_handler)
    LOGGER.addHandler(debug_log_handler)
    
OUTPUT_DIR = 'grand_hsic_test_outputs/{}/'.format(datetime.datetime.now().isoformat())
DEBUG_LOG_PATH = OUTPUT_DIR + 'debug_log'
INFO_LOG_PATH = OUTPUT_DIR + 'info_log'
WARNING_LOG_PATH = OUTPUT_DIR + 'warning_log'

configure_logger()
os.makedirs(OUTPUT_DIR)
add_file_handlers(WARNING_LOG_PATH, INFO_LOG_PATH, DEBUG_LOG_PATH)

In [7]:
import traceback

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.tsa.ar_model as ar
import torch
torch.set_default_dtype(torch.double)
import sigkernel_scale
import sigkernel
import warnings

In [9]:
import pprint
pp = pprint.PrettyPrinter(indent = 4)

In [10]:
import scipy.stats as stats

In [11]:
from tqdm import tqdm

In [12]:
from itertools import product

In [13]:
import matplotlib.pyplot as plt

In [14]:
from scipy.optimize import brentq

In [15]:
#SIG_KERNEL_SAMPLES_NO = 100
#SAMPLE_LENGTHS = [3, 6, 11, 31, 101]
#M_RUNS = 50
#CORRELATIONS = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.8]
#EXTINCTION_RATES = [0, 0.5, 1.]
#RBF_SIGMAS = [0.5, 1., 4., 6., 8.]
#DYADIC_ORDERS = [0, 2, 4]
#A=[0., 0.2, 0.5, 0.8]

SIG_KERNEL_SAMPLES_NO = 5
SAMPLE_LENGTHS = [3]
M_RUNS = 2
CORRELATIONS = [0, 0.1]
EXTINCTION_RATES = [0, 0.5]
RBF_SIGMAS = [0.5]
DYADIC_ORDERS = [0]
A=[0.]

In [16]:
KERNELS_NORMALIZED = [ (sigkernel_scale.LinearKernel(), None) ] 
KERNELS_NORMALIZED += [ (sigkernel_scale.RBFKernel(sigma), sigma) for sigma in RBF_SIGMAS ]
KERNELS_UNNORMALIZED = [ (sigkernel.LinearKernel(), None) ] 
KERNELS_UNNORMALIZED += [ (sigkernel.RBFKernel(sigma), sigma) for sigma in RBF_SIGMAS ]

In [17]:
LOGGER.setLevel('DEBUG')
OVERFLOWS = 0

In [18]:
def psi(x, M=4, a=1):
    if x <= np.sqrt(M):
        return x ** 2
    return M + M ** (1 + a) * (M ** (-a) - x ** (-2*a)) / a

In [19]:
def get_postscale_for_batch_adaptive_dyadic_order(paths, static_kernel):
    LOGGER.debug('FINDING POSTSCALE FOR BATCH')
    
    postscale = torch.zeros(paths.shape[0])
    for i in range(paths.shape[0]):
        dyadic_order = 0
        warnings.filterwarnings("error", category = RuntimeWarning)

        while True:
            try:
                kernel = sigkernel_scale.SigKernel(static_kernel, dyadic_order)
                kernel_xx = kernel.compute_kernel(paths[i:i+1, :, :], paths[i:i+1, :, :])
                target = psi(np.sqrt(kernel_xx.item()), 1e298, 1e-10)
                f = lambda lam: target - kernel.compute_kernel(paths[i:i+1, :, :], paths[i:i+1, :, :], torch.tensor([lam]), torch.tensor([lam])).item()
                lam = min(max(brentq(f, 1e-300, 1 - 1e-300, xtol=1e-300, maxiter = 1000), 0), 1)
                postscale[i] = lam
                break
            except (ValueError, RuntimeWarning) as e:
                if str(e) == 'overflow encountered in double_scalars':
                    global OVERFLOWS
                    LOGGER.debug(str(e))
                    LOGGER.debug(traceback.format_exc())
                    OVERFLOWS += 1
                dyadic_order += 1
            
        warnings.resetwarnings()
            
    return postscale, dyadic_order

In [20]:
def run_experiment_for_unnormalized_kernels(proc_gen_fct):
    results = []
    
    for kernel, kernel_param in KERNELS_UNNORMALIZED:
        for run in range(M_RUNS):
            for order in DYADIC_ORDERS:
                LOGGER.info('Experiment for unnormalized kernel: run {}, order {}, kernel param {}'.format(run, order, kernel_param))
                
                processes = proc_gen_fct()
                X_raw = processes[:, :, 0]
                Y_raw = processes[:, :, 1]
                X_lead_lag_chopped = torch.stack((X_raw[:, 1:], X_raw[:, :-1]), axis = 2)
                Y_lead_lag_chopped = torch.stack((Y_raw[:, 1:], Y_raw[:, :-1]), axis = 2)

                null, hsic_score, p_val = hsic_independence_test.get_test_result_matrix_shuffle(X_lead_lag_chopped, Y_lead_lag_chopped, static_kernel = kernel, dyadic_order = order)
                
                LOGGER.info('Result p_val: {}'.format(p_val))
                
                results.append({
                    'kernel_type' : 'unnormalized',
                    'kernel_param' : kernel_param,
                    'dyadic_order' : order,
                    'hsic_score' : hsic_score,
                    'p_val' : p_val,
                })
    
    return results

In [21]:
def run_experiment_for_normalized_kernels(proc_gen_fct):
    results = []
    
    for kernel, kernel_param in KERNELS_NORMALIZED:
        for run in range(M_RUNS):
            for order in DYADIC_ORDERS:
                LOGGER.info('Experiment for normalized kernel: run {}, order {}, kernel param {}'.format(run, order, kernel_param))
                
                processes = proc_gen_fct()
                X_raw = processes[:, :, 0]
                Y_raw = processes[:, :, 1]
                X_lead_lag_chopped = torch.stack((X_raw[:, 1:], X_raw[:, :-1]), axis = 2)
                Y_lead_lag_chopped = torch.stack((Y_raw[:, 1:], Y_raw[:, :-1]), axis = 2)

                postscale_x, order_x = get_postscale_for_batch_adaptive_dyadic_order(X_lead_lag_chopped, kernel);
                postscale_y, order_y = get_postscale_for_batch_adaptive_dyadic_order(Y_lead_lag_chopped, kernel);

                null, hsic_score, p_val = hsic_independence_test.get_test_result_matrix_shuffle(X_lead_lag_chopped, Y_lead_lag_chopped, static_kernel = kernel, dyadic_order = order, postscale_x = postscale_x, postscale_y = postscale_y)
                
                LOGGER.info('Result p_val: {}'.format(p_val))
                
                results.append({
                    'kernel_type' : 'normalized',
                    'kernel_param' : kernel_param,
                    'dyadic_order' : order,
                    'hsic_score' : hsic_score,
                    'p_val' : p_val,
                })
    
    return results

In [22]:
def push_results(results):
    LOGGER.debug('Pushing results')
    LOGGER.debug('Current results on stack: ' + str(push_results.paths))
    if len(push_results.paths) == 5: 
        oldest_results_file = push_results.paths.pop(0)
        os.remove(oldest_results_file)
        
    output_file_path = OUTPUT_DIR + 'intermediate_results_{}.pickle'.format(datetime.datetime.now().isoformat()).replace(' ', '_').replace(':', '.')        
    
    with open(output_file_path, 'wb') as f:
        pickle.dump(results, f)
    
    LOGGER.debug('Saved ' + output_file_path)
    push_results.paths.append(output_file_path)

push_results.paths = []

In [23]:
results = []
for sl in SAMPLE_LENGTHS:
    for a in A:
        for ext_rate in EXTINCTION_RATES:
            LOGGER.info('============ STARTING ANOTHER RUN : EXTINCT GAUSSIAN ==============')
            LOGGER.info('Sample length: {}, A: {}, extinction rate: {}'.format(sl, a, ext_rate))

            proc_gen_fct = lambda: gen_processes.gen_ar_iid_samples_extinct_gaussian_burn_in(sl, SIG_KERNEL_SAMPLES_NO, a=a, extinction_rate=ext_rate)
            results_norm = run_experiment_for_normalized_kernels(proc_gen_fct)
            results_unnorm = run_experiment_for_unnormalized_kernels(proc_gen_fct)
            
            for result in results_norm + results_unnorm:
                result['process_type'] = 'extinct_gaussian'
                result['a'] = a
                result['process_param'] = ext_rate

            results += results_norm + results_unnorm
            
            push_results(results)
            LOGGER.debug('============ FINISHED ANOTHER RUN : EXTINCT GAUSSIAN ==============')
        for corr in CORRELATIONS:
            LOGGER.debug('============ STARTING ANOTHER RUN : LINEAR CORRELATION ==============')
            LOGGER.info('Sample length: {}, A: {}, correlation: {}'.format(sl, a, corr))

            proc_gen_fct = lambda: gen_processes.gen_ar_iid_samples_burn_in(sl, SIG_KERNEL_SAMPLES_NO, a=a, corr=corr)
            results_norm = run_experiment_for_normalized_kernels(proc_gen_fct)
            results_unnorm = run_experiment_for_unnormalized_kernels(proc_gen_fct)
            
            for result in results_norm + results_unnorm:
                result['process_type'] = 'lin_corr'
                result['a'] = a
                result['process_param'] = ext_rate

            results += results_norm + results_unnorm
            
            push_results(results)
            LOGGER.debug('============ FINISHED ANOTHER RUN : LINEAR CORRELATION ==============')

        

============ STARTING ANOTHER RUN : EXTINCT GAUSSIAN ==============
Sample length: 3, A: 0.0, extinction rate: 0
Experiment for normalized kernel: run 0, order 0, kernel param None
FINDING POSTSCALE FOR BATCH
FINDING POSTSCALE FOR BATCH


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 2107.12it/s]

Result p_val: 0.1
Experiment for normalized kernel: run 1, order 0, kernel param None
FINDING POSTSCALE FOR BATCH
FINDING POSTSCALE FOR BATCH



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 3244.71it/s]

Result p_val: 0.46
Experiment for normalized kernel: run 0, order 0, kernel param 0.5


...

Result p_val: 0.02
Experiment for unnormalized kernel: run 0, order 0, kernel param 0.5



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 3024.36it/s]

Result p_val: 0.34
Experiment for unnormalized kernel: run 1, order 0, kernel param 0.5



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 2742.85it/s]

Result p_val: 0.7
Pushing results
Current results on stack: ['grand_hsic_test_outputs/2022-04-12T14:20:37.252521/intermediate_results_2022-04-12T14.20.39.504661.pickle', 'grand_hsic_test_outputs/2022-04-12T14:20:37.252521/intermediate_results_2022-04-12T14.20.40.588928.pickle', 'grand_hsic_test_outputs/2022-04-12T14:20:37.252521/intermediate_results_2022-04-12T14.20.41.572461.pickle']
Saved grand_hsic_test_outputs/2022-04-12T14:20:37.252521/intermediate_results_2022-04-12T14.20.42.547433.pickle
============ FINISHED ANOTHER RUN : LINEAR CORRELATION ==============


In [24]:
results_final = {
    'results' : results,
    'hyperparams' : {
        'SIG_KERNEL_SAMPLES_NO' : SIG_KERNEL_SAMPLES_NO,
        'SAMPLE_LENGTHS' : SAMPLE_LENGTHS,
        'M_RUNS' : M_RUNS,
        'CORRELATIONS' : CORRELATIONS,
        'EXTINCTION_RATES' : EXTINCTION_RATES,
        'RBF_SIGMAS' : RBF_SIGMAS,
        'DYADIC_ORDERS' : DYADIC_ORDERS,
        'A': A
    },
    'overflows' : OVERFLOWS
}
output_file_path = OUTPUT_DIR + 'results_final.pickle'
with open(output_file_path, 'wb') as f:
    pickle.dump(results_final, f)


In [25]:
pp.pprint(results_final)

{   'hyperparams': {   'A': [0.0],
                       'CORRELATIONS': [0, 0.1],
                       'DYADIC_ORDERS': [0],
                       'EXTINCTION_RATES': [0, 0.5],
                       'M_RUNS': 2,
                       'RBF_SIGMAS': [0.5],
                       'SAMPLE_LENGTHS': [3],
                       'SIG_KERNEL_SAMPLES_NO': 5},
    'overflows': 0,
    'results': [   {   'a': 0.0,
                       'dyadic_order': 0,
                       'hsic_score': 69.03863433977719,
                       'kernel_param': None,
                       'kernel_type': 'normalized',
                       'p_val': 0.1,
                       'process_param': 0,
                       'process_type': 'extinct_gaussian'},
                   {   'a': 0.0,
                       'dyadic_order': 0,
                       'hsic_score': 5.798053892546919,
                       'kernel_param': None,
                       'kernel_type': 'normalized',
                       '

                       'kernel_type': 'unnormalized',
                       'p_val': 0.3,
                       'process_param': 0.5,
                       'process_type': 'lin_corr'},
                   {   'a': 0.0,
                       'dyadic_order': 0,
                       'hsic_score': 1.2740885280864078,
                       'kernel_param': 0.5,
                       'kernel_type': 'unnormalized',
                       'p_val': 0.18,
                       'process_param': 0.5,
                       'process_type': 'lin_corr'},
                   {   'a': 0.0,
                       'dyadic_order': 0,
                       'hsic_score': 1.1245461854234904,
                       'kernel_param': None,
                       'kernel_type': 'normalized',
                       'p_val': 0.78,
                       'process_param': 0.5,
                       'process_type': 'lin_corr'},
                   {   'a': 0.0,
                       'dyadic_order': 0,
        